In [33]:
############################### Importation du fichier de données #######################################
import os
os.chdir('/Users/learegazzetti/Documents/Cours/M1_Info/Data_Mining_M1_Info/Projet')

import pandas
df = pandas.read_csv('data_avec_etiquettes.txt', sep = '\t')


#Vérifications de la cohérence des données
print(df.shape)
print(df.head())
print(df.info())


#outil pour mesurer le temps
import timeit
print(dir(timeit))

(494021, 200)
   V1  V2  V3  V4  V5  V6  V7  V8  V9  V10  ...  V191  V192  V193  V194  V195  \
0   0   0   1   5   0   8   0   1   5    1  ...     9   1.0   0.0  0.11   0.0   
1   0   0   1   1   1   7   0   1   8    1  ...    19   1.0   0.0  0.05   0.0   
2   1   0   0   9   0   2   0   0   3    1  ...    29   1.0   0.0  0.03   0.0   
3   0   0   0   5   0   3   1   0   0    0  ...    39   1.0   0.0  0.03   0.0   
4   0   1   0   9   1   4   0   1   4    1  ...    49   1.0   0.0  0.02   0.0   

   V196  V197  V198  V199  V200  
0   0.0   0.0   0.0   0.0   m12  
1   0.0   0.0   0.0   0.0   m12  
2   0.0   0.0   0.0   0.0   m12  
3   0.0   0.0   0.0   0.0   m12  
4   0.0   0.0   0.0   0.0   m12  

[5 rows x 200 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494021 entries, 0 to 494020
Columns: 200 entries, V1 to V200
dtypes: float64(15), int64(181), object(4)
memory usage: 753.8+ MB
None
['Timer', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', 

In [34]:
############################### Exploitation d’un processeur multicœur avec H2O ############################
#importation de la librairie H2O
import h2o
#vérification de version
print(h2o.__version__)

#démarrage de la machine "h2o"
h2o.init(nthreads = -1)



3.32.0.2
Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "11" 2018-09-25; Java(TM) SE Runtime Environment 18.9 (build 11+28); Java HotSpot(TM) 64-Bit Server VM 18.9 (build 11+28, mixed mode)
  Starting server from /Users/learegazzetti/opt/anaconda3/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/mv/sjmp9nkn3ps2fjvtx4mk81s80000gn/T/tmpqrpf_g9_
  JVM stdout: /var/folders/mv/sjmp9nkn3ps2fjvtx4mk81s80000gn/T/tmpqrpf_g9_/h2o_learegazzetti_started_from_python.out
  JVM stderr: /var/folders/mv/sjmp9nkn3ps2fjvtx4mk81s80000gn/T/tmpqrpf_g9_/h2o_learegazzetti_started_from_python.err
  Server is running at http://127.0.0.1:54329
Connecting to H2O server at http://127.0.0.1:54329 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Europe/Paris
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.2
H2O_cluster_version_age:,1 month and 10 days
H2O_cluster_name:,H2O_from_python_learegazzetti_c3r53l
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


In [35]:
############################### Subdivision des données en échantillon test (test) et échantillon d'apprentissage (train)
import sklearn
from sklearn import model_selection
 
train,test = model_selection.train_test_split(df,test_size = round(df.shape[0]*30/100),random_state=1) 
#test_size=30% de 494021 
#random_state fixé à 1 pour pouvoir reproduire l'approche 

print(train.shape,test.shape)


train_h2o = h2o.H2OFrame(train)
test_h2o = h2o.H2OFrame(test)


(345815, 200) (148206, 200)
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [37]:
############################# Régression logistique multinomiale parce que la variable cible V200 possède 23 modalités

#importation de la régression généralisée
from h2o.estimators import H2OGeneralizedLinearEstimator
#instanciation - régression logistique multinomiale
modelH2o = H2OGeneralizedLinearEstimator(family="multinomial")

#modélisation
#apprentissage -- mesure du temps de calcul
depart = timeit.default_timer()
modelH2o.train(x=train_h2o.columns[0:-1:],y="V200",training_frame=train_h2o)
duree = timeit.default_timer()- depart
print(str(duree)+" sec.")


/Users/learegazzetti/opt/anaconda3/lib/python3.8/site-packages/h2o/estimators/estimator_base.py:200: RuntimeWarning: Dropping bad and constant columns: [V179, V178]
  warnings.warn(mesg["message"], RuntimeWarning)


glm Model Build progress: |███████████████████████████████████████████████| 100%
5538.9969850239995 sec.


In [5]:
#affichage par défaut
modelH2o.show()


Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  GLM_model_python_1609070993222_1


GLM Model: summary


,,family,link,regularization,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
0,,multinomial,multinomial,"Elastic Net (alpha = 0.5, lambda = 9.882E-4 )",6302,104,30,Key_Frame__upload_81938391d7dc0915d304b4e2d9814956.hex




ModelMetricsMultinomialGLM: glm
** Reported on train data. **

MSE: 0.004571306724697665
RMSE: 0.06761143930355029

Scoring History: 


,,timestamp,duration,iterations,negative_log_likelihood,objective,training_rmse,training_logloss,training_r2,training_classification_error
0,,2020-12-27 13:11:30,0.000 sec,0,373196.520092,1.079180,,,,
1,,2020-12-27 13:15:00,3 min 29.824 sec,1,35896.034889,0.118287,0.116453,0.103801,0.999191,0.0124257
2,,2020-12-27 13:17:39,6 min 9.069 sec,2,20875.896844,0.080829,0.0896672,0.0602216,0.99952,0.00933736
3,,2020-12-27 13:20:14,8 min 43.404 sec,3,12623.759069,0.058689,0.0862845,0.0351042,0.999556,0.00828767
4,,2020-12-27 13:22:45,11 min 15.007 sec,4,9482.740532,0.052077,0.0730246,0.0271642,0.999682,0.00566488
5,,2020-12-27 13:25:08,13 min 37.707 sec,5,9410.494007,0.049817,0.0695738,0.0251096,0.999711,0.00507497
6,,2020-12-27 13:27:12,15 min 42.323 sec,6,8464.182557,0.047184,0.0691617,0.0242268,0.999715,0.00483206
7,,2020-12-27 13:29:20,17 min 49.779 sec,7,8352.745870,0.046500,0.0685679,0.0239053,0.999719,0.00443301
8,,2020-12-27 13:32:10,20 min 40.373 sec,8,8374.922409,0.046264,0.0684203,0.0239754,0.999721,0.00440698
9,,2020-12-27 13:34:43,23 min 12.515 sec,9,8305.261521,0.046152,0.0682916,0.0237728,0.999722,0.0043636



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,V194,2.939942,1.000000,0.087097
1,V181,2.724798,0.926820,0.080723
2,V180,2.188166,0.744289,0.064825
3,V187,1.875135,0.637814,0.055552
4,V166,1.631881,0.555073,0.048345
5,V191,1.607437,0.546758,0.047621
6,V168,1.606357,0.546391,0.047589
7,V170,1.580410,0.537565,0.046820
8,V190,1.570953,0.534348,0.046540
9,V193,1.517309,0.516102,0.044951



See the whole table with table.as_data_frame()


In [6]:
#affichage complet des coefficients avec les tests de significativité
modelH2o.pprint_coef()



Coefficients: glm multinomial coefficients


,names,coefs_class_0,coefs_class_1,coefs_class_2,coefs_class_3,coefs_class_4,coefs_class_5,coefs_class_6,coefs_class_7,coefs_class_8,...,std_coefs_class_13,std_coefs_class_14,std_coefs_class_15,std_coefs_class_16,std_coefs_class_17,std_coefs_class_18,std_coefs_class_19,std_coefs_class_20,std_coefs_class_21,std_coefs_class_22
0,Intercept,-5.102720,-0.837175,-5.057877,1.262039,-12.060512,-11.655047,-5.329336,-6.771507,-11.367365,...,-7.930472,-5.241374,-7.659087,-8.007982,-7.198426,-10.451074,-5.667364,-9.288605,-10.9619,-11.144221
1,V161.m1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
2,V161.m10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
3,V161.m11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
4,V161.m12,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
5,V161.m13,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
6,V161.m14,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
7,V161.m15,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
8,V161.m16,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
9,V161.m17,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.453904,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000



See the whole table with table.as_data_frame()



In [7]:
#Test de significativite globale du modele
#stat. test du rapport de vraisemblance
LR = modelH2o.null_deviance() - modelH2o.residual_deviance() 
print(LR)

729976.3874244669


In [8]:
#degrés de liberté
ddl = modelH2o.null_degrees_of_freedom() - modelH2o.residual_degrees_of_freedom()
print(ddl)

126


In [9]:
#librairie scipy
import scipy.stats as stats
#p-value de significativité globale
pvalue = 1.0 - stats.chi2.cdf(x=LR,df=ddl)
print(pvalue) #0.0 => Le modèle est globalement très significatif

0.0


In [10]:
#affichage matrice de confusion
modelH2o.confusion_matrix(train_h2o)



Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,m1,m10,m11,m12,m13,m14,m15,m16,m17,m18,...,m23,m3,m4,m5,m6,m7,m8,m9,Error,Rate
0,1483.0,0.0,0.0,55.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.035761,55 / 1 538
1,0.0,75029.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000133,10 / 75 039
2,0.0,0.0,0.0,154.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,1.000000,163 / 163
3,17.0,18.0,0.0,68009.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,4.0,0.0,42.0,0.0,0.0,0.0,0.002391,163 / 68 172
4,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,2 / 2
5,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,3 / 3
6,0.0,0.0,0.0,27.0,0.0,0.0,143.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,13.0,0.0,0.0,0.0,0.218579,40 / 183
7,0.0,70.0,0.0,53.0,0.0,0.0,0.0,498.0,0.0,107.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.315934,230 / 728
8,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,4 / 4
9,0.0,14.0,0.0,133.0,0.0,0.0,0.0,0.0,0.0,968.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.136485,153 / 1 121



See the whole table with table.as_data_frame()


In [11]:
#prédiction sur l'échantillon test - format H2o obtenu
predm_h2o = modelH2o.predict(test_h2o)
#retro-convertir en Pandas
ppredm_h2o = predm_h2o.as_data_frame()
print(ppredm_h2o.head())

glm prediction progress: |████████████████████████████████████████████████| 100%


/Users/learegazzetti/opt/anaconda3/lib/python3.8/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset column 'V161' has levels not trained on: ["m45"]
  warnings.warn(w)


  predict        m1       m10       m11       m12           m13           m14  \
0     m19  0.000155  0.000062  0.000058  0.000496  5.319101e-08  7.978651e-08   
1     m19  0.000248  0.000099  0.000093  0.000983  8.485699e-08  1.272855e-07   
2     m10  0.000127  0.998665  0.000076  0.000259  6.369574e-08  9.554361e-08   
3     m19  0.000155  0.000062  0.000058  0.000496  5.319101e-08  7.978651e-08   
4     m19  0.000155  0.000062  0.000058  0.000496  5.319101e-08  7.978651e-08   

        m15       m16           m17  ...       m21       m22       m23  \
0  0.000045  0.000014  1.063820e-07  ...  0.000003  0.000026  0.000004   
1  0.000071  0.000023  1.697140e-07  ...  0.000005  0.000041  0.000007   
2  0.000053  0.000014  1.273915e-07  ...  0.000004  0.000058  0.000005   
3  0.000045  0.000014  1.063820e-07  ...  0.000003  0.000026  0.000004   
4  0.000045  0.000014  1.063820e-07  ...  0.000003  0.000026  0.000004   

         m3        m4            m5        m6            m7         

In [12]:
#matrice de confusion en test -- avec le seuil de H20
pandas.crosstab(test["V200"],ppredm_h2o.predict)


predict,m1,m10,m12,m15,m16,m18,m19,m2,m21,m22,m23,m4,m6,m7
V200,,,,,,,,,,,,,,
m1,3,130,131,0,1,0,364,0,0,1,0,0,0,0
m10,47,2599,2505,6,15,39,7001,1,28,8,0,1,28,1
m11,0,16,10,0,0,0,42,0,0,0,0,0,0,0
m12,82,4686,4090,16,28,69,11932,0,42,18,1,3,55,2
m13,0,0,0,0,0,0,1,0,0,0,0,0,0,0
m14,0,0,1,0,0,0,0,0,0,0,0,0,0,0
m15,0,9,6,0,0,0,14,0,0,0,0,0,0,0
m16,0,52,28,0,0,1,109,0,1,0,0,0,0,0
m17,0,0,2,0,0,0,2,0,0,0,0,0,0,0


In [13]:
#scikit-learn
from sklearn import metrics
#taux d'erreur
print(1-metrics.accuracy_score(test["V200"],ppredm_h2o.predict)) #0.004122640109037401


0.004122640109037401


In [14]:
############################# Selection de variables

#Contribution des variables dans le modele
contribution_variables = modelH2o._model_json['output']['variable_importances'].as_data_frame()
#contribution_variables[contribution_variables.relative_importance > 0]
print(contribution_variables)


    variable  relative_importance  scaled_importance  percentage
0       V194             2.939942           1.000000    0.087097
1       V181             2.724798           0.926820    0.080723
2       V180             2.188166           0.744289    0.064825
3       V187             1.875135           0.637814    0.055552
4       V166             1.631881           0.555073    0.048345
..       ...                  ...                ...         ...
268     V157             0.000000           0.000000    0.000000
269     V158             0.000000           0.000000    0.000000
270     V173             0.000000           0.000000    0.000000
271     V175             0.000000           0.000000    0.000000
272     V176             0.000000           0.000000    0.000000

[273 rows x 4 columns]


In [16]:
var_sel = contribution_variables.variable[contribution_variables.relative_importance>1]
print(var_sel)

0     V194
1     V181
2     V180
3     V187
4     V166
5     V191
6     V168
7     V170
8     V190
9     V193
10    V182
11    V195
12    V192
Name: variable, dtype: object


In [17]:
print(len(var_sel))

13


In [31]:
new_train = pandas.concat([train[var_sel.values],train['V200']],axis=1)
new_train_h2o = h2o.H2OFrame(new_train)

new_test = pandas.concat([test[var_sel.values],test['V200']],axis=1)
new_test_h2o = h2o.H2OFrame(new_test)


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [32]:
print(new_train.shape)
print(new_test.shape)

(345815, 14)
(148206, 14)


In [19]:
#importation de la régression généralisée
from h2o.estimators import H2OGeneralizedLinearEstimator
#instanciation - régression logistique multinomiale
new_modelH2o = H2OGeneralizedLinearEstimator(family="multinomial")


In [20]:
#modélisation
#apprentissage -- mesure du temps de calcul
depart = timeit.default_timer()
new_modelH2o.train(x=new_train_h2o.columns[0:-1:],y="V200",training_frame=new_train_h2o)
duree = timeit.default_timer()- depart
print(str(duree)+" sec.")


glm Model Build progress: |███████████████████████████████████████████████| 100%


/Users/learegazzetti/opt/anaconda3/lib/python3.8/site-packages/h2o/job.py:70: UserWarning: Reached maximum number of iterations 50!
  warnings.warn(w)


953.7958849259994 sec.


In [21]:
#affichage par défaut
new_modelH2o.show()

Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  GLM_model_python_1609070993222_2


GLM Model: summary


,,family,link,regularization,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
0,,multinomial,multinomial,"Elastic Net (alpha = 0.5, lambda = 9.882E-4 )",322,54,50,Key_Frame__upload_b285e41a61fc082f6460eb1330104d18.hex




ModelMetricsMultinomialGLM: glm
** Reported on train data. **

MSE: 0.007812481803338331
RMSE: 0.08838824471239562

Scoring History: 


,,timestamp,duration,iterations,negative_log_likelihood,objective,training_rmse,training_logloss,training_r2,training_classification_error
0,,2020-12-27 14:51:47,0.000 sec,0,373196.520092,1.079180,,,,
1,,2020-12-27 14:52:14,27.582 sec,1,42347.626505,0.133971,0.129266,0.122457,0.999003,0.0146928
2,,2020-12-27 14:52:31,44.519 sec,2,18755.617274,0.074152,0.105546,0.0540851,0.999335,0.0136865
3,,2020-12-27 14:52:47,1 min 0.248 sec,3,13969.498812,0.061686,0.0900315,0.0401552,0.999516,0.00718014
4,,2020-12-27 14:53:02,1 min 15.257 sec,4,13192.142693,0.060033,,,,
5,,2020-12-27 14:53:16,1 min 29.538 sec,5,13160.357281,0.059483,0.0881881,0.0377634,0.999536,0.00718881
6,,2020-12-27 14:53:34,1 min 47.067 sec,6,13088.325254,0.059244,0.0881138,0.037555,0.999537,0.00719171
7,,2020-12-27 14:53:49,2 min 2.100 sec,7,13150.993258,0.059116,,,,
8,,2020-12-27 14:54:08,2 min 21.059 sec,8,13071.003534,0.059032,0.0883153,0.0375056,0.999535,0.00720327
9,,2020-12-27 14:54:24,2 min 37.104 sec,9,13122.848248,0.058971,0.0884742,0.0376539,0.999533,0.00720327



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,V194,3.734160,1.000000,0.127307
1,V181,3.665759,0.981682,0.124975
2,V187,3.185969,0.853196,0.108618
3,V193,2.584731,0.692185,0.088120
4,V170,2.512690,0.672893,0.085664
5,V180,2.417365,0.647365,0.082414
6,V191,2.187667,0.585853,0.074583
7,V166,1.806614,0.483807,0.061592
8,V168,1.694012,0.453653,0.057753
9,V190,1.650896,0.442106,0.056283


In [22]:
#affichage complet des coefficients avec les tests de significativité
new_modelH2o.pprint_coef()



Coefficients: glm multinomial coefficients


,names,coefs_class_0,coefs_class_1,coefs_class_2,coefs_class_3,coefs_class_4,coefs_class_5,coefs_class_6,coefs_class_7,coefs_class_8,...,std_coefs_class_13,std_coefs_class_14,std_coefs_class_15,std_coefs_class_16,std_coefs_class_17,std_coefs_class_18,std_coefs_class_19,std_coefs_class_20,std_coefs_class_21,std_coefs_class_22
0,Intercept,-6.432795,2.621662,-5.525733,-0.691605,-12.060512,-11.655047,-5.906241,-5.313284,-11.367365,...,-8.555608,-5.732598,-9.981071,-11.144221,-9.227299,-10.451074,-5.934581,-10.18871,-10.9619,-11.144221
1,V194,-0.808775,-2.165101,0.000000,-2.021662,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0000,0.000000
2,V181,0.000000,0.001122,0.000000,-0.007094,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0000,0.000000
3,V180,0.000000,0.000000,0.000000,47.605525,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.670995,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0000,0.000000
4,V187,0.000000,-5.158159,0.000000,1.325129,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0000,0.000000
5,V166,0.000000,0.000000,0.000000,-1.893575,0.000000,0.000000,4.839787,0.000000,0.000000,...,0.915220,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0000,0.000000
6,V191,0.000000,-0.004963,0.000000,0.006762,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-0.450665,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0000,0.000000
7,V168,0.560024,0.000000,0.000000,-1.645392,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0000,0.000000
8,V170,2.335332,-1.152713,0.000000,0.717331,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.801575,0.000000,0.000000,0.000000,0.000000,-0.092053,0.00000,0.0000,0.000000
9,V190,0.006229,0.007433,0.000000,-0.000462,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-0.172106,0.000000,0.000000,0.000000,0.000000,-0.037901,0.00000,0.0000,0.000000


In [23]:
#Test de significativite globale du modele
#stat. test du rapport de vraisemblance
LR = new_modelH2o.null_deviance() - new_modelH2o.residual_deviance() 
print(LR)

720573.7782070863


In [24]:
#degrés de liberté
ddl = new_modelH2o.null_degrees_of_freedom() - new_modelH2o.residual_degrees_of_freedom()
print(ddl)

76


In [25]:
#librairie scipy
import scipy.stats as stats
#p-value de significativité globale
pvalue = 1.0 - stats.chi2.cdf(x=LR,df=ddl)
print(pvalue) #0.0 => Le modèle est globalement très significatif


0.0


In [26]:
#prédiction sur l'échantillon test - format H2o obtenu
new_predm_h2o = new_modelH2o.predict(new_test_h2o)
#retro-convertir en Pandas
new_ppredm_h2o = new_predm_h2o.as_data_frame()
print(new_ppredm_h2o.head())


glm prediction progress: |████████████████████████████████████████████████| 100%
  predict        m1       m10       m11       m12           m13           m14  \
0     m19  0.000175  0.000078  0.000061  0.000430  8.810481e-08  1.321572e-07   
1     m19  0.000286  0.000121  0.000099  0.000946  1.437675e-07  2.156513e-07   
2     m10  0.000146  0.996569  0.000078  0.001727  1.019603e-07  1.529405e-07   
3     m19  0.000175  0.000078  0.000061  0.000430  8.810481e-08  1.321572e-07   
4     m19  0.000175  0.000078  0.000061  0.000430  8.810481e-08  1.321572e-07   

        m15       m16           m17  ...       m21       m22           m23  \
0  0.000041  0.000060  1.762096e-07  ...  0.000003  0.000024  7.048385e-07   
1  0.000068  0.000098  2.875350e-07  ...  0.000005  0.000039  1.150140e-06   
2  0.000048  0.000109  2.039207e-07  ...  0.000003  0.000076  8.156826e-07   
3  0.000041  0.000060  1.762096e-07  ...  0.000003  0.000024  7.048385e-07   
4  0.000041  0.000060  1.762096e-07  ...  

In [27]:
#matrice de confusion en test -- avec le seuil de H20
pandas.crosstab(new_test["V200"],new_ppredm_h2o.predict)


predict,m1,m10,m12,m15,m18,m19,m21,m22,m6
V200,,,,,,,,,
m1,3,132,131,0,0,364,0,0,0
m10,49,2611,2510,5,36,7002,28,6,32
m11,0,16,10,0,0,42,0,0,0
m12,84,4702,4121,13,57,11929,42,15,61
m13,0,0,0,0,0,1,0,0,0
m14,0,0,1,0,0,0,0,0,0
m15,0,9,6,0,0,14,0,0,0
m16,0,52,28,0,1,109,1,0,0
m17,0,0,2,0,0,2,0,0,0


In [28]:
#scikit-learn
from sklearn import metrics
#taux d'erreur
print(1-metrics.accuracy_score(new_test["V200"],new_ppredm_h2o.predict)) #0.007347880652605165

#Certes on a augmenté le taux d'erreur, mais il reste du même ordre de grandeur
#Et surtout on est passé de 274 variables à 58


#éteindre le moteur "h2o"
h2o.cluster().shutdown()

0.007347880652605165
H2O session _sid_b7a9 closed.
